# Scientific Computing with Python (Second Edition)
# Chapter 15

We start by importing all from Numpy. As explained in Chapter 01 the examples are written assuming this import is initially done.

In [1]:
from numpy import *

### 15.1 Manual testing
No code.

## 15.2 Automatic testing
### 15.2.1 Testing the bisection algorithm

In [2]:
def bisect(f, a, b, tol=1.e-8):
    """
    Implementation of the bisection algorithm 
    f real valued function
    a,b interval boundaries (float) with the property 
    f(a) * f(b) <= 0
    tol tolerance (float)
    """
    if f(a) * f(b)> 0:
        raise ValueError("Incorrect initial interval [a, b]") 
    for i in range(100):
        c = (a + b) / 2.
        if f(a) * f(c) <= 0:
            b = c
        else:
            a = c
        if abs(a - b) < tol:
            return (a + b) / 2
    raise Exception('No root found within the given tolerance {tol}')

In [3]:
def test_identity():
    result = bisect(lambda x: x, -1., 1.) 
    expected = 0.
    assert allclose(result, expected),'expected zero not found'

test_identity()

In [4]:
def test_badinput():
    try:
        bisect(lambda x: x,0.5,1)
    except ValueError:
        pass
    else:
        raise AssertionError()

test_badinput()

In [5]:
def test_equal_boundaries():
    result = bisect(lambda x: x, 0., 0.)
    expected = 0.
    assert allclose(result, expected), \
                   'test equal interval bounds failed'

def test_reverse_boundaries():
    result = bisect(lambda x: x, 1., -1.)
    expected = 0.
    assert allclose(result, expected),\
                 'test reverse int_erval bounds failed'
 
test_equal_boundaries()
test_reverse_boundaries()


### 15.2.2 Using the unittest module

The code examples differ here slightly from those in the book because of the notebook environment.
The difference is in the call 
`unittest.main(argv=[''], verbosity=2, exit=False)`

In [6]:
from bisection import bisect
import unittest

class TestIdentity(unittest.TestCase):
    def test(self):
        result = bisect(lambda x: x, -1.2, 1.,tol=1.e-8)
        expected = 0.
        self.assertAlmostEqual(result, expected)

if __name__=='__main__':
    unittest.main(argv=[''], verbosity=2, exit=False)

test (__main__.TestIdentity) ... ok

----------------------------------------------------------------------
Ran 1 test in 0.001s

OK


In [7]:
from bisection import bisect
import unittest

class TestIdentity(unittest.TestCase):
    def test(self):
        result = bisect(lambda x: x, -1.2, 1.,tol=1.e-3)
        expected = 0.
        self.assertAlmostEqual(result, expected)

if __name__=='__main__':
    unittest.main(argv=[''], verbosity=2, exit=False)

test (__main__.TestIdentity) ... FAIL

FAIL: test (__main__.TestIdentity)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-7-f3dde6271355>", line 8, in test
    self.assertAlmostEqual(result, expected)
AssertionError: 0.00017089843750002018 != 0.0 within 7 places (0.00017089843750002018 difference)

----------------------------------------------------------------------
Ran 1 test in 0.001s

FAILED (failures=1)


In [8]:
import unittest
from bisection import bisect

class TestIdentity(unittest.TestCase):
    def identity_fcn(self,x):
        return x
    def test_functionality(self):
        result = bisect(self.identity_fcn, -1.2, 1.,tol=1.e-8)
        expected = 0.
        self.assertAlmostEqual(result, expected)
    def test_reverse_boundaries(self):
        result = bisect(self.identity_fcn, 1., -1.)
        expected = 0.
        self.assertAlmostEqual(result, expected)
    def test_exceeded_tolerance(self):
        tol=1.e-80
        self.assertRaises(Exception, bisect, self.identity_fcn,
                                               -1.2, 1.,tol)
if __name__=='__main__':
    unittest.main(argv=[''], verbosity=2, exit=False)

test_exceeded_tolerance (__main__.TestIdentity) ... ok
test_functionality (__main__.TestIdentity) ... ok
test_reverse_boundaries (__main__.TestIdentity) ... ok

----------------------------------------------------------------------
Ran 3 tests in 0.002s

OK


### 15.2.3 Test setUp and tearDown methods

In [9]:
class StringNotFoundException(Exception):
    pass

def find_string(file, string):
    for i,lines in enumerate(file.readlines()):
        if string in lines:
            return i
    raise StringNotFoundException(
          f'String {string} not found in File {file.name}.')

In [10]:
import unittest
import os # used for, for example, deleting files

from find_in_file import find_string, StringNotFoundException

class TestFindInFile(unittest.TestCase):
    def setUp(self):
        file = open('test_file.txt', 'w')
        file.write('bird')
        file.close()
        self.file = open('test_file.txt', 'r')
    def tearDown(self):
        self.file.close()
        os.remove(self.file.name)
    def test_exists(self):
        line_no=find_string(self.file, 'bird')
        self.assertEqual(line_no, 0)
    def test_not_exists(self):
        self.assertRaises(StringNotFoundException, find_string,
                                              self.file, 'tiger')

if __name__=='__main__':
    unittest.main(argv=[''], verbosity=2, exit=False)

test_exists (__main__.TestFindInFile) ... ok
test_not_exists (__main__.TestFindInFile) ... ok
test_exceeded_tolerance (__main__.TestIdentity) ... ok
test_functionality (__main__.TestIdentity) ... ok
test_reverse_boundaries (__main__.TestIdentity) ... ok

----------------------------------------------------------------------
Ran 5 tests in 0.003s

OK


In [11]:
class Tests(unittest.TestCase):
    def checkifzero(self,fcn_with_zero,interval):
        result = bisect(fcn_with_zero,*interval,tol=1.e-8)
        function_value=fcn_with_zero(result)
        expected=0.
        self.assertAlmostEqual(function_value, expected)

In [12]:
test_data=[
           {'name':'identity', 'function':lambda x: x,
                                     'interval' : [-1.2, 1.]},
           {'name':'parabola', 'function':lambda x: x**2-1,
                                        'interval' :[0, 10.]},
           {'name':'cubic', 'function':lambda x: x**3-2*x**2,
                                       'interval':[0.1, 5.]},
               ] 
def make_test_function(dic):
    return lambda self :\
        self.checkifzero(dic['function'],dic['interval'])
for data in test_data:
    setattr(Tests, f"test_{data['name']}", make_test_function(data))

if __name__=='__main__': 
    unittest.main(argv=[''], verbosity=2, exit=False)

test_exists (__main__.TestFindInFile) ... ok
test_not_exists (__main__.TestFindInFile) ... ok
test_exceeded_tolerance (__main__.TestIdentity) ... ok
test_functionality (__main__.TestIdentity) ... ok
test_reverse_boundaries (__main__.TestIdentity) ... ok
test_cubic (__main__.Tests) ... ok
test_identity (__main__.Tests) ... ok
test_parabola (__main__.Tests) ... ok

----------------------------------------------------------------------
Ran 8 tests in 0.006s

OK


### 15.2.5 Assertion tools
No code.
### 15.2.6 Float comparisons

In [13]:
import numpy.linalg as nla
A=random.rand(10,10)
[Q,R]=nla.qr(A)

In [14]:
import numpy.testing as npt 
npt.assert_allclose(
               Q.T @ Q,identity(Q.shape[0]),atol=1.e-12)

In [15]:
import numpy.testing as npt
npt.assert_allclose(Q @ R,A)

In [16]:
import unittest
import numpy.testing as npt
from numpy.linalg import qr

class TestQR(unittest.TestCase):
    def setUp(self):
        self.A=random.rand(10,10)
        [self.Q,self.R]=qr(self.A)
    def test_orthogonal(self):
        npt.assert_allclose(
            self.Q.T @ self.Q,identity(self.Q.shape[0]),
            atol=1.e-12)
    def test_sanity(self):
            npt.assert_allclose(self.Q @ self.R,self.A)

if __name__=='__main__':
    unittest.main(argv=[''], verbosity=2, exit=False)

test_exists (__main__.TestFindInFile) ... ok
test_not_exists (__main__.TestFindInFile) ... ok
test_exceeded_tolerance (__main__.TestIdentity) ... ok
test_functionality (__main__.TestIdentity) ... ok
test_reverse_boundaries (__main__.TestIdentity) ... ok
test_orthogonal (__main__.TestQR) ... ok
test_sanity (__main__.TestQR) ... ok
test_cubic (__main__.Tests) ... ok
test_identity (__main__.Tests) ... ok
test_parabola (__main__.Tests) ... ok

----------------------------------------------------------------------
Ran 10 tests in 0.007s

OK


###  15.2.7 Unit and functional tests

In [17]:
def bisect_step(f, a, b, n):
    """
    Implementation of the bisection algorithm
    f real valued function
    a,b interval boundaries (float) with the property
    f(a) * f(b) <= 0
    tol tolerance (float)
    """
    for iteration in range(n):
        if f(a) * f(b)> 0:
            raise ValueError("Incorrect initial interval [a, b]")
        c = (a + b) / 2.
        if f(a) * f(c) <= 0:
            b = c
        else:
            a = c
    return a,b

In [18]:
import unittest

class TestMidpoint(unittest.TestCase):
    def identity_fcn(self,x):
        return x
    def test_midpoint(self):
        a,b = bisect_step(self.identity_fcn,-2.,1.,1)
        self.assertAlmostEqual(a,-0.5)
        self.assertAlmostEqual(b,1)
if __name__=='__main__':
    unittest.main(argv=[''], verbosity=2, exit=False)

test_exists (__main__.TestFindInFile) ... ok
test_not_exists (__main__.TestFindInFile) ... ok
test_exceeded_tolerance (__main__.TestIdentity) ... ok
test_functionality (__main__.TestIdentity) ... ok
test_reverse_boundaries (__main__.TestIdentity) ... ok
test_midpoint (__main__.TestMidpoint) ... ok
test_orthogonal (__main__.TestQR) ... ok
test_sanity (__main__.TestQR) ... ok
test_cubic (__main__.Tests) ... ok
test_identity (__main__.Tests) ... ok
test_parabola (__main__.Tests) ... ok

----------------------------------------------------------------------
Ran 11 tests in 0.012s

OK


### 15.2.8 Debugging

In [19]:
test_case = TestIdentity(methodName='test_reverse_boundaries')

In [20]:
test_case.debug()

## 15.3 Measuring execution time
### 15.3.1 Timing with a magic function

In [21]:
A=zeros((1000,1000))
A[53,67]=10

def find_elements_1(A):
    b = []
    n, m = A.shape
    for i in range(n):
        for j in range(m):
            if abs(A[i, j]) > 1.e-10:
                b.append(A[i, j])
    return b

def find_elements_2(A):
    return [a for a in A.reshape((-1, )) if abs(a) > 1.e-10]

def find_elements_3(A):
    return [a for a in A.flatten() if abs(a) > 1.e-10]
 
def find_elements_4(A):
    return A[where(0.0 != A)]

In [22]:
%timeit -n 50 -r 3 find_elements_1(A)

257 ms ± 123 µs per loop (mean ± std. dev. of 3 runs, 50 loops each)


In [23]:
%timeit -n 50 -r 3 find_elements_2(A)

191 ms ± 406 µs per loop (mean ± std. dev. of 3 runs, 50 loops each)


In [24]:
%timeit -n 50 -r 3 find_elements_3(A)

205 ms ± 5.67 ms per loop (mean ± std. dev. of 3 runs, 50 loops each)


In [25]:
%timeit -n 50 -r 3 find_elements_4(A)

1.78 ms ± 22.4 µs per loop (mean ± std. dev. of 3 runs, 50 loops each)


### 15.3.2 Timing with the Python module timeit


In [26]:
import timeit
setup_statements="""
from scipy import zeros
from numpy import where
A=zeros((1000,1000))
A[57,63]=10.

def find_elements_1(A):
    b = []
    n, m = A.shape
    for i in range(n):
        for j in range(m):
            if abs(A[i, j]) > 1.e-10:
               b.append(A[i, j])
    return b

def find_elements_2(A):
    return [a for a in A.reshape((-1,)) if abs(a) > 1.e-10]

def find_elements_3(A):
    return [a for a in A.flatten() if abs(a) > 1.e-10]

def find_elements_4(A):
    return A[where( 0.0 != A)]
"""
experiment_1 = timeit.Timer(stmt = 'find_elements_1(A)',
                            setup = setup_statements)
experiment_2 = timeit.Timer(stmt = 'find_elements_2(A)',
                            setup = setup_statements)
experiment_3 = timeit.Timer(stmt = 'find_elements_3(A)',
                            setup = setup_statements)
experiment_4 = timeit.Timer(stmt = 'find_elements_4(A)',
                            setup = setup_statements)

In [27]:
t1 = experiment_1.repeat(3,5) 
t2 = experiment_2.repeat(3,5) 
t3 = experiment_3.repeat(3,5) 
t4 = experiment_4.repeat(3,5) 
# Results per loop in ms
min(t1)*1000/5 # 615 ms
min(t2)*1000/5 # 543 ms
min(t3)*1000/5 # 546 ms
min(t4)*1000/5 # 7.26 ms

1.903121208306402

### 15.3.3 Timing with a context manager

In [28]:
import time
class Timer:
    def __enter__(self):
        self.start = time.time()
        # return self
    def __exit__(self, ty, val, tb):
        end = time.time()
        self.elapsed=end-self.start
        print(f'Time elapsed {self.elapsed} seconds') 
        return False

In [29]:
with Timer():
  find_elements_1(A)

Time elapsed 0.2798018455505371 seconds


In [30]:
with Timer():
  find_elements_2(A)

Time elapsed 0.23056364059448242 seconds


In [31]:
with Timer():
  find_elements_3(A)

Time elapsed 0.21068072319030762 seconds


In [32]:
with Timer():
  find_elements_4(A)

Time elapsed 0.002699613571166992 seconds
